<center><strong>Sistem Temu Kembali Informasi</strong><br />
<strong><font color="blue">Semester Gasal T.A. 2020/2021</font></strong><br />
</center>

<strong>Outline pertemuan minggu ke-4</strong><br />
<li> Word Embedding </li>

### Word Embedding

<p> Representasi dari teks ke dalam dense matriks. Term yang memiliki makna serupa berada pada jarak yang berdekatan pada ruang vektor </p>
<p> VSM merepresentasikan teks ke dalam sparse matriks </p>
<p> Sparse matriks memiliki ukuran yang besar dan memiliki banyak nilai 0 </p>
<p> Dense matriks ukurannya lebih kecil, dapat diinisialisasi, dan tidak memiliki nilai 0 </p>
<h2><img alt="" src="figures/3_word_embeddings.png" style="height: 296px ; width: 602px" /></h2>

#### Algoritma word embedding
<ul>
    <li> Skip-gram </li>
    <li> CBoW (Continuous Bag of Words)</li>
    <li> GloVe (Global Vector)</li>
    <li> FastText </li>
    <li> BERT (based on context) </li>
</ul>

#### Skip Gram dan CBoW
<h2><img alt="" src="figures/sg-cbow.png"/></h2>

<p> Dimana w(t) merupakan kata target dan w(t-n) - w(t+n) adalah konteks (kata yang berada disekitar kata target. n merupakan ukuran dari window size. Apabila n = 2 maka konteks didefinisikan sebagai 2 kata sebelum dan setelah kata target.</p>
<p> Contoh: "the big brown bear and the fox"
    <br>Apabila kata target adalah = "brown" dan n = 2 
    <br>Maka: 
    <br>w(t) = "brown"
    <br>w(t-2) = "the"
    <br>w(t-1) = "big"
    <br>w(t+1) = "bear"
    <br>w(t+2) = "and"
</p>
    

In [27]:
# from google colab
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'/P4
!ls

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/P4
data  figures  model  Pertemuan_4.ipynb


In [28]:
import os
import pickle
from sklearn.datasets import fetch_20newsgroups

In [29]:
# download 3 categoies from 20newgroup dataset
# save it as binary

categories = ['sci.med', 'talk.politics.misc',  'rec.autos']
data = fetch_20newsgroups(subset='train', categories=categories,\
                          remove=('headers', 'footers', 'quotes'))

dst_name = "20newsgroup.pckl"
dst_path = os.path.join("data", dst_name)
with open(dst_path, 'wb') as fout:
    pickle.dump(data, fout)

In [30]:
# loada data pickle
with open(dst_path, 'rb') as fin:
    data = pickle.load(fin)

data = [doc for doc in data.data]

In [31]:
data[6]

'\nOk boys and girls,\n\n"What was the \'Ogadan War\'????"\n\nThe Money Raised in Band-Aid covered How Much of\nthe Cost of Which Soviet Client State to replace what\ncatagory of weapon system lost in the aforementioned war?\n\nWhy was the Joke: "We arm the World." Really Not that funny?\n\nGonzo Station is the designation for WHICH USN Op Area?\nand the primary threat targets in the Area Were:.....\n\nciao\ndrieux\n\n\n'

In [32]:
# preprocessing
import re
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from nltk import word_tokenize

def preprocess(doc):
    sents = sent_tokenize(doc)
    sents_tok = list() # tokenisasi kalimat
    for s in sents:
        s = s.strip().lower() # case folding dan menghilangkan new line
        s = s.replace("\n", " ") # menggantikan \n dengan spasi
        s = re.sub(r'[^a-zA-Z0-9 ]', ' ', s) # menghapus simbol
        s = re.sub(' +', ' ', s) # menghapus repetitive space
        
        sents_tok.append(s)
    
    return " ".join(sents_tok)

docs = list()
for d in data:
    docs.append(preprocess(d))
#docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
# Untuk membentuk word embedding digunakan library Gensim
# Format inputan dari Gensim adalah list of words untuk setiap kalimat
# Contoh pada dokumen:
#       Tiger is the biggest cat alive. The tigers species can be divided into 5 groups.
# Inputan yang dibutuhkan Gensim adalah:
#       ["tiger", "is", "the", "biggest", "cat", "alive"], ["the", "tigers", "species", 
#        "can", "be", "devided", "into", "5", "groups"]
#
doc_gensim = [word_tokenize(d) for d in docs]
#print(doc_gensim)

In [34]:
# train skip-gram dengan library gensim
#
from gensim.models import Word2Vec

d = 300
model_sg = Word2Vec(doc_gensim, min_count=1, size=d, window=5, workers=-1)
print("DONE")

DONE


In [35]:
import os

model_path = os.path.join('model', 'model_sg.model')
# model dari word embedding dapat disimpan dengan fungsi di bawah ini
model_sg.save(model_path)

# untuk menggunakan model yang telah disimpan sebelumnya dapat dilakukan
# dengan memanfaatkan fungsi load, beberapa model membutuhkan waktu yang lama
# karena memiliki ukuran yang besar
model_sg_load = Word2Vec.load(model_path)

print('DONE')


DONE


#### Word2 Vec menggunakan matriks dense

<p>Penggunaan memori oleh Gensim</p>
<p>Jumlah kata x size x 12 bytes <br>
    Contoh: <br>
    Jika terdapat 100.000 kata unik dengan menggunakan dimensi embedding 200 <br>
    Maka memori yang dibutuhkan = 100.000 x 200 x 12 bytes = ~229MB
</p>

In [36]:
# Melihat vektor suatu kata
model_sg.wv['car']

array([ 9.2163071e-04,  1.1693257e-04, -1.5775987e-03, -1.1942146e-03,
       -1.4069499e-03,  7.0930464e-04,  9.9954254e-04, -1.6399580e-03,
       -4.0395552e-04,  9.0569491e-04, -1.5023064e-04, -9.8585826e-04,
        1.1478326e-03,  7.6774799e-04,  8.7342726e-04, -7.5985387e-05,
       -9.2184790e-05,  6.6724991e-05, -5.9735484e-04, -1.1551884e-03,
        3.7036341e-04, -8.4486842e-04, -6.6639506e-04,  1.0475286e-03,
        4.4623867e-04, -1.6348661e-03,  7.1942998e-04, -4.8876496e-04,
        1.2077856e-03,  8.5683155e-04, -1.2063410e-03,  7.7154573e-06,
        1.1511529e-03, -1.4446944e-03, -1.1662329e-03,  1.5370405e-03,
        1.2719422e-03, -4.7268887e-04,  6.6855014e-04, -2.0495088e-04,
       -1.3005155e-03, -1.0767892e-03,  1.6657349e-03, -1.5025460e-03,
        3.8901065e-04, -1.6350874e-03,  7.4762240e-05, -4.3703528e-04,
       -1.1248885e-03, -1.3902395e-03, -9.1467868e-04,  4.6449443e-04,
        1.5605160e-03, -5.3503609e-04, -1.2790418e-04, -2.7422924e-04,
      

In [37]:
# Menghitung similarity vektor antar kata

model_sg.wv.similarity('car', 'car')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


1.0

In [38]:
#  Menampilkan top-N similar words

model_sg.wv.similar_by_word('car', topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('gatos', 0.20472154021263123),
 ('wpi', 0.204578697681427),
 ('eck', 0.2040053755044937),
 ('pampered', 0.2019251435995102),
 ('drivability', 0.2005893588066101),
 ('margoli', 0.1942407190799713),
 ('scarce', 0.1886296570301056),
 ('prying', 0.18846502900123596),
 ('syntex', 0.1884564459323883),
 ('books', 0.1880016177892685)]

In [39]:
# Mencari yang paling mirip dengan kata 'cars'
model_sg.wv.most_similar('cars')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('caveats', 0.23042982816696167),
 ('uvietta', 0.21939630806446075),
 ('goran', 0.21160390973091125),
 ('pass', 0.20260515809059143),
 ('stealths', 0.20066936314105988),
 ('spin', 0.20015880465507507),
 ('carroll', 0.19842290878295898),
 ('ingunale', 0.18859432637691498),
 ('vulnerability', 0.18656852841377258),
 ('hull', 0.1844136118888855)]

<p><img alt="" src="figures/3_cosine.png" style="height:400px; width:683px" /></p>

In [40]:
# error jika kata tidak ada di training data

kata = 'entir'
try:
    print(model_sg.wv.most_similar(kata))
except:
    print('error! kata "',kata,'" tidak ada di training data')

error! kata " entir " tidak ada di training data


### Note: ###
<ul>
    <li> Pada word2vec apabila suatu kata tidak terdapat dalam list vocabulary maka akan error </li>
    <li> Kesalahan pengetikan (typo) bisa meninmbulkan error </li>
    <li> Untuk mangatasi hal ini dapat dilakukan dengan menggunakan algoritma lain, seperti FastText </li>

In [41]:
# "predict" vector for new data without re-training from the beginning
d1 = ['new','generation','nvidia','gpu','is','rtx']
d2 = ['deep','learning','computation','mostly', 'on', 'gpu']
d3 = ['the','rtx','gpu','capable','super','sampling','ssdl']
D = [d1,d2,d3]
model_sg.train(D, total_examples=len(D), epochs=model_sg.epochs)
print(model_sg.predict_output_word('gpu'))

[('intrigues', 4.4517652e-05), ('enduring', 4.4517652e-05), ('cardinal', 4.4517652e-05), ('shirer', 4.4517652e-05), ('chancellors', 4.4517652e-05), ('undermining', 4.4517652e-05), ('waning', 4.4517652e-05), ('strasser', 4.4517652e-05), ('schleicher', 4.4517652e-05), ('unite', 4.4517652e-05)]


<h3 id="Latihan:"><font color="blue">Latihan 1:</font></h3>

<ul> 
    <li> Bangunlah word embedding dengan data 3 kategori lainnya dari data 20newsGroup</li>
    <li> Lakukan training menggunakan Skip-Gram dan CBoW </li>
    <li> Apakah vektor yang dihasilkan oleh suatu kata sama? ketika menggunakan Skip-Gram dan CBoW </li>
    <li> Apakah hasil top-n similar memperoleh vocabulary yang sama? </li>
    <li> Apakah jumlah dimensi berpengaruh pada hasil top-n most similar? </li>

[link](https://machinelearningmastery.com/develop-word-embeddings-python-gensim/)

### Bangunlah word embedding dengan data 3 kategori lainnya dari data 20newsGroup

In [42]:
# Kerjakan latihan 1 pada cell berikut ini
categories = ['sci.space', 'talk.politics.guns',  'rec.motorcycles']
data = fetch_20newsgroups(subset='train', categories=categories,\
                          remove=('headers', 'footers', 'quotes'))

dst_name = "20newsgroup_lat1.pckl"
dst_path = os.path.join("data", dst_name)
with open(dst_path, 'wb') as fout:
    pickle.dump(data, fout)
# loada data pickle
with open(dst_path, 'rb') as fin:
    data = pickle.load(fin)

data = [doc for doc in data.data]

In [43]:
docs = list()
for d in data:
    docs.append(preprocess(d))
doc_gensim = [word_tokenize(d) for d in docs]

### Lakukan training menggunakan Skip-Gram dan CBoW

#### Skip-Gram

In [44]:
# train skip-gram dengan library gensim
d = 300
model_sg_l1 = Word2Vec(doc_gensim, min_count=1, size=d, window=5, workers=-1,\
                       sg=1)

#### CBoW

In [45]:
# train CBoW dengan library gensim
model_cbow_l1 = Word2Vec(doc_gensim, min_count=1, size=d, window=5, workers=-1)

### Apakah vektor yang dihasilkan oleh suatu kata sama ketika menggunakan Skip-Gram dan CBoW? Sama

In [56]:
import numpy as np
np.testing.assert_array_equal(model_sg_l1.wv['sun'], model_sg_l1.wv['sun'], \
                              err_msg="beda meski identik")

In [64]:
for vocab in list(model_sg_l1.wv.vocab):
  np.testing.assert_array_equal(model_sg_l1.wv[vocab], model_cbow_l1.wv[vocab],\
                              err_msg=f"Vektor kata {vocab} yang dihasilkan tak\
                              sama")

### Apakah hasil top-n similar memperoleh vocabulary yang sama? Sama

In [73]:
np.testing.assert_array_equal(
    model_sg_l1.wv.similar_by_word('sun', topn=10),
    model_sg_l1.wv.similar_by_word('sun', topn=10),
    err_msg="beda meski identik")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [74]:
for vocab in list(model_sg_l1.wv.vocab):
  np.testing.assert_array_equal(
    model_sg_l1.wv.similar_by_word(vocab, topn=10),
    model_cbow_l1.wv.similar_by_word(vocab, topn=10),
    err_msg=f"Vektor kata {vocab} yang dihasilkan tak sama")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Apakah jumlah dimensi berpengaruh pada hasil top-n most similar? Berpengaruh

In [75]:
# train skip-gram dengan library gensim
d = 100
model_sg_l1_d100 = Word2Vec(doc_gensim, min_count=1, size=d, window=5, \
                            workers=-1, sg=1)

In [81]:
for vocab in list(model_sg_l1.wv.vocab):
  np.testing.assert_array_equal(
    model_sg_l1.wv.similar_by_word(vocab, topn=10),
    model_sg_l1_d100.wv.similar_by_word(vocab, topn=10),
    err_msg=f"Vektor kata {vocab} yang dihasilkan tak sama")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


AssertionError: ignored

In [82]:
 model_sg_l1.wv.similar_by_word('sun', topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('money', 0.26274335384368896),
 ('ps', 0.2186214029788971),
 ('mega', 0.21569541096687317),
 ('lennon', 0.20447425544261932),
 ('xs1100', 0.20339816808700562),
 ('matt', 0.202156201004982),
 ('restored', 0.2015211135149002),
 ('recovery', 0.198196679353714),
 ('wheate', 0.19777661561965942),
 ('sweden', 0.19638556241989136)]

In [83]:
 model_sg_l1_d100.wv.similar_by_word('sun', topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('throng', 0.42842212319374084),
 ('speedo', 0.4172268509864807),
 ('wheate', 0.37708377838134766),
 ('perijove', 0.36823323369026184),
 ('jem', 0.36290794610977173),
 ('defuse', 0.3624386489391327),
 ('resumes', 0.3574535846710205),
 ('cookson', 0.35729891061782837),
 ('compression', 0.3562437891960144),
 ('uncertified', 0.34896132349967957)]

## FastText (Facebook-2016)##
<ul>
    <li> Menggunakan Sub-words: app, ppl, ple - apple, sehingga dapat mengatasi permasalahan typo atau tidak adanya suatu kata dalam vocabularies </li>
    <li> Paper: https://arxiv.org/abs/1607.04606  </li>
    <li> Website: https://fasttext.cc/ </li>
    <li> Source: https://github.com/facebookresearch/fastText </li>
</ul>

In [84]:
from gensim.models import FastText

dim = 300 # Jumlah neurons = ukuran vektor = jumlah kolom
model_ft = FastText(doc_gensim, size=dim, window=5, min_count=2, workers=-1)
'Done'

'Done'

In [85]:
#  Menampilkan top-N similar words

model_ft.wv.similar_by_word('car', topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('care', 0.3797631561756134),
 ('carbon', 0.371961772441864),
 ('card', 0.3593623638153076),
 ('carbs', 0.3565341532230377),
 ('oscar', 0.3559572100639343),
 ('cards', 0.35226595401763916),
 ('carry', 0.3510536551475525),
 ('carl', 0.3446604311466217),
 ('carb', 0.34046387672424316),
 ('carriers', 0.33848267793655396)]

In [86]:
# Mencari yang paling mirip dengan kata 'cars'
model_ft.wv.most_similar('cars')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('scars', 0.4224168360233307),
 ('mars', 0.3457052707672119),
 ('carriers', 0.3412468135356903),
 ('care', 0.3377683758735657),
 ('wars', 0.32996058464050293),
 ('car', 0.32129740715026855),
 ('careers', 0.3060256838798523),
 ('quasars', 0.29644346237182617),
 ('bars', 0.2964199185371399),
 ('carbs', 0.28946223855018616)]

In [87]:
# Melihat similarity antar kata

print(model_ft.wv.similarity('cars', 'car'))
print(model_sg.wv.similarity('cars', 'car'))

0.3212974
0.06379849


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [88]:
# Tidak error jika kata tidak ada di training data

kata = 'beckmans'
try:
    print(model_ft.wv.most_similar(kata))
except:
    print('error! kata "',kata,'" tidak ada di training data')
    

# Tidak terjadi error saat kata tidak terdapat pada vocabulary

[('humans', 0.3696466088294983), ('germans', 0.355217307806015), ('blackman', 0.3220236897468567), ('jackman', 0.319011390209198), ('beans', 0.3016563653945923), ('ans', 0.2911129891872406), ('means', 0.26861149072647095), ('sans', 0.26726704835891724), ('leans', 0.25095224380493164), ('bans', 0.2482897937297821)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


<h3 id="Latihan:"><font color="blue">Latihan 2:</font></h3>

<ul>
	<li>Apakah kelebihan dan kekurangan WE secara umum?</li>
	<li>Apakah kira-kira aplikasi yang dapat memanfaatkan WE?</li>
	<li>Apakah bisa dijadikan representasi dokumen? Bagaimana caranya?</li>
	<li>Bergantung pada apa sajakah performa model WE?</li>
</ul>

* Preprocessing apa yang sebaiknya dilakukan pada model Word Embedding?
* Apakah Pos Tag bermanfaat disini? Jika iya bagaimana menggunakannya?

### Kerjakan latihan 2 pada cell berikut ini

* Apakah kelebihan dan kekurangan WE secara umum?
  - kekurangan [link](https://aylien.com/blog/word-embeddings-and-their-challenges)
    - ketidakmampuan mengatasi kata diluar kamus (OOV)
    - tak ada representasi terbagi pada level sub-kata
    - scaling ke bahasa baru butuh matriks embedding baru
    - tak dapat digunakan untuk inisiasi arsitektur SOTA
  - kelebihan [link](https://aylien.com/blog/word-embeddings-and-their-challenges)
    - penyusunan angka merepresentasikan informasi semantik dan sintaksis dari kata dan konteks mereka
    - tak hanya belajar arti dan semantik dari kata, namun juga sintak dan grammar pada tingkatan tertentu
    - tak perlu anotasi yang berlebihan
    - dapat diturunkan dari corpus tak beranotasi yang dapat dibaca
    - pre-trained embedding dapat digunakan pada tugas yang menggunakan data berlabel sedikit
* Apakah kira-kira aplikasi yang dapat memanfaatkan WE?
  - aplikasi NLP secara umum yang membutuhkan pemahaman bahasa seperti Analisis Respon Survei, Analisis Komentar, dan Sistem Rekomendasi Music/Video [link](https://www.kdnuggets.com/2019/02/word-embeddings-nlp-applications.html)
* Apakah bisa dijadikan representasi dokumen? Bagaimana caranya?
  - bisa, dengan memadukan pendekatan vektor dokumen (melalui model Vector Space Model) dengan word embedding untuk mengukur tingkat kemiripan [link](http://xplordat.com/2018/09/27/word-embeddings-and-document-vectors-part-1-similarity/)
* Bergantung pada apa sajakah performa model WE?
  - jumlah corpus kata dan konteks yang digunakan, bahasa yang digunakan dan pre-processing kata sebelum di olah menjadi matriks
* Preprocessing apa yang sebaiknya dilakukan pada model Word Embedding?
  - tak semua tugas butuh level pre-pemrosesan yang sama. Berikut general rule of thumb berdasarkan data dan domain [link](https://www.kdnuggets.com/2019/04/text-preprocessing-nlp-machine-learning.html)  
  ![grot](https://cdn-images-1.medium.com/max/1600/1*VRgrWGLBJJDXsQE72QyI3g.png)
* Apakah Pos Tag bermanfaat disini? Jika iya bagaimana menggunakannya?
  - bisa bermanfaat [link](https://stackoverflow.com/questions/51537441/how-to-combine-both-word-embeddings-and-pos-embedding-together-to-build-the-clas)
  - berasumsi konteks tiap kata itu penting:
    - komputasikan word embedding
    - komputasikan POS
    - jalankan LSTM atau recurrent layer yang sejenis pada POS (tak wajib)
    - tiap kata buat representasi berisikan word embedding diteruskan oleh keluaran yang terkait dari LSTM layer
    - gunakan FC layer untuk membuat hidden representation yang konsisten